In [ ]:
%%capture

%cd ../../

In [2]:
import sys
sys.path.extend(["kaggle-persona_identifier/src"])

In [3]:
import pandas as pd
import os
from pathlib import Path
import json

In [4]:
current_dir = Path.cwd()
raw_data_dir = current_dir.joinpath('kaggle-persona_identifier/data/raw')
processed_data_dir = current_dir.joinpath('kaggle-persona_identifier/data/processed')
config_dir = current_dir.joinpath('kaggle-persona_identifier/configs')
artifacts_dir = current_dir.joinpath('kaggle-persona_identifier/models/artifacts')
features_dir = current_dir.joinpath('kaggle-persona_identifier/models/features')
filename = 'kaggle_survey_2017_2021.csv'

In [5]:
full_data_dir = processed_data_dir.joinpath('full-data')
train_dir =  processed_data_dir.joinpath('train-data')
test_dir = processed_data_dir.joinpath('test-data')
models_dir = current_dir.joinpath('model/artifacts/')

In [6]:
test_data = pd.read_csv(test_dir.joinpath('KaggleResponses-Test.csv'))
test_data.shape

(31870, 37)

In [9]:
test_sample = test_data.iloc[:,:-1][0:1]
json_data = json.dumps(test_sample.to_dict(orient='records'))
json_dat= {"test_data": json_data}

In [12]:
test_sample.shape

(1, 36)

In [10]:
import json
with open('test_data.json', 'w') as f:
    json.dump(json_data, f)

In [11]:
test_sample.columns

Index(['What is your current yearly compensation (approximate $USD)?',
       'Uses Computer Vision Algorithms Count',
       'What is the highest level of formal education that you have attained or plan to attain within the next 2 years?_Doctorate',
       'Uses ML Framework Count', 'Uses ML Algorithms Count',
       'Which of the following big data products (relational database, data warehouse, data lake, or similar) do you use most often? - Selected Choice',
       'What type of computing platform do you use most often for your data science projects? - Selected Choice',
       'In what industry is your current employer/contract (or your most recent employer if retired)? - Selected Choice',
       'For how many years have you used machine learning methods?',
       'Uses NLP Algorithms Count',
       'What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - Javascript',
       'Which types of specialized hardware do you use on a regular ba

In [8]:
json_data

'[{"What is your current yearly compensation (approximate $USD)?": 1.0, "Uses Computer Vision Algorithms Count": 0.0, "What is the highest level of formal education that you have attained or plan to attain within the next 2 years?_Doctorate": 0.0, "Uses ML Framework Count": 0.0, "Uses ML Algorithms Count": 0.0, "Which of the following big data products (relational database, data warehouse, data lake, or similar) do you use most often? - Selected Choice": 0.0, "What type of computing platform do you use most often for your data science projects? - Selected Choice": 0.0, "In what industry is your current employer/contract (or your most recent employer if retired)? - Selected Choice": 0.0, "For how many years have you used machine learning methods?": 0.0, "Uses NLP Algorithms Count": 0.0, "What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - Javascript": 0.0, "Which types of specialized hardware do you use on a regular basis?  (Select all t

In [9]:
len(json_data)

14799

In [4]:
import json
from model import test_model, convert


ModuleNotFoundError: No module named 'src'

In [ ]:
def handler(event, context):
    """
    Lambda function handler for predicting linguistic acceptability of the given sentence
    :param event: post/get request context
    :param context: Context of Meta data
    """
    if "resource" in event.keys():
        body = event["body"]
        body = json.loads(body)
        test_df = pd.DataFrame(body)
        prediction = test_model(test_df)
        response = convert(prediction)
        return {
            "statusCode": 200,
            "headers": {},
            "body": json.dumps(response)
        }
    else:
        body = json.loads(event['test_data'])
        test_df = pd.DataFrame(body)
        prediction = test_model(test_df)
        response = convert(prediction)
        return response


if __name__ == "__main__":
    test = json_dat
    result = handler(test, None)
    print(result)

In [1]:

## Build docker image

In [4]:
import os
os.chdir('/home/ec2-user/SageMaker/persona_identifier/')

In [5]:
os.getcwd()

'/home/ec2-user/SageMaker/persona_identifier'

In [6]:
!docker images

REPOSITORY   TAG       IMAGE ID   CREATED   SIZE


In [7]:
!docker rmi -f  9d97d58341d  dangling = true

Error: No such image: 9d97d58341d
Error: No such image: dangling
Error response from daemon: invalid reference format
Error: No such image: true


In [1]:
!docker build -t 505184942745.dkr.ecr.us-east-1.amazonaws.com/kaggle-persona-identifier:dev .

unable to prepare context: unable to evaluate symlinks in Dockerfile path: lstat /home/ec2-user/SageMaker/persona_identifier/notebooks/Dockerfile: no such file or directory


In [51]:
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 505184942745.dkr.ecr.us-east-1.amazonaws.com

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [52]:
!docker push 505184942745.dkr.ecr.us-east-1.amazonaws.com/kaggle-persona-identifier:dev

The push refers to repository [505184942745.dkr.ecr.us-east-1.amazonaws.com/kaggle-persona-identifier]

a1ab5726: Preparing 
7f38f717: Preparing 
220ed00c: Preparing 
02664824: Preparing 
27e40e2e: Preparing 
33d1c8af: Preparing 
6c63f3a2: Preparing 
f6bb007b: Preparing 
77e4650a: Preparing 
220ed00c: Pushed   543.3MB/543.1MBAdev: digest: sha256:0ba19177cc9aeab7e388072d3006e79c30541bae9b8f435c3b6d91916629e3e8 size: 2428


# Danger Zone: Donot Run them or Touch it If you do not Understand

In [ ]:
# !aws lambda update-function-code \
#     --function-name  kaggle-persona-identifier \
#     --image-uri 505184942745.dkr.ecr.us-east-1.amazonaws.com/kaggle-persona-identifier:dev

In [19]:
# !aws lambda update-function-configuration \
#     --function-name kaggle-persona-identifier \
#     --memory-size 1024

In [18]:
# !aws lambda get-function \
#     --function-name  kaggle-persona-identifier

# Danger zone Ended